In [1]:
import math

def get_angle(point1, point2, point3):
    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    
    angle1 = math.atan2(y2 - y1, x2 - x1)
    angle2 = math.atan2(y3 - y2, x3 - x2)
    
    angle = abs(math.degrees(angle2 - angle1))
    return angle

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/755.1 kB 25.3 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8x-pose-p6.pt', verbose=False)  # load an official model
# model = YOLO('path/to/best.pt')  # load a custom model

# Predict with the model
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image

100%|██████████| 190M/190M [00:03<00:00, 51.4MB/s] 


100%|██████████| 476k/476k [00:00<00:00, 30.5MB/s]


image 1/1 /kaggle/working/bus.jpg: 1280x960 4 persons, 94.6ms
Speed: 15.5ms preprocess, 94.6ms inference, 3233.9ms postprocess per image at shape (1, 3, 1280, 960)


In [ ]:
import cv2
from ultralytics import YOLO
import pandas as pd
import numpy as np

# Load the YOLOv8 model
# model = YOLO('yolov8n.pt')
video_path = '/kaggle/input/pune-metro-hackathon/dataset/escalator-fall/1.avi'
# Open the video file
cap = cv2.VideoCapture(video_path)
frame_count = 0
threshold_angle = 50
df = pd.DataFrame(columns=["person_number", "frame_number", "side", "angle"])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        if frame_count % 1 == 0:  # Skip 10 frames
            neccessary_points = [12, 14, 16, 11, 13, 15]
            results = model(frame, stream=False)  # predict on an image
            print(frame_count)

            # Loop through all keypoints
            r=results[0]
            x_y_points = []
            for person_number in range(len(r.keypoints[0].xy)):
                if len(r.keypoints[0].xy[person_number]) != 17:
                    print('keypoint empty')
                    continue
                # Get the x and y coordinates of the keypoint
                for point_number in neccessary_points:
                    x = int(r.keypoints[0].xy[person_number][point_number][0])
                    y = int(r.keypoints[0].xy[person_number][point_number][1])
                    if x == 0:
                        print(f'{frame_count}, empty, x:{x}, y:{y}')
                        x_y_points = []
                        break
                    x_y_points.append((x, y))
                    # Draw a circle at the keypoint location with the random color
                    # cv2.circle(image2, (int(x), int(y)), 5, color, -1)
                if len(x_y_points) == 0:
                    continue
                # print(x_y_points)
                right_angle = get_angle(x_y_points[0], x_y_points[1], x_y_points[2])
                left_angle = get_angle(x_y_points[3], x_y_points[4], x_y_points[5])

                if right_angle > threshold_angle and left_angle > threshold_angle:
                    print(f'{frame_count}, BENT ? {right_angle}, {left_angle}')
                    df = pd.concat([df, pd.DataFrame({'person_number': person_number, 'frame_number': frame_count, 'side': ['right', 'left'], 'angle': [right_angle, left_angle]})], ignore_index=True)
                    continue
                # Append side and angle to df dataframe

    else:
        # Break the loop if the end of the video is reached
        break
    
    # Read the next frame
    frame_count += 1

# Release the video capture object and close the display window
cap.release()
df.to_csv('df.csv')


0: 768x1280 1 person, 82.7ms
Speed: 7.2ms preprocess, 82.7ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)
0

0: 768x1280 1 person, 75.6ms
Speed: 5.6ms preprocess, 75.6ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
1

0: 768x1280 1 person, 75.5ms
Speed: 6.1ms preprocess, 75.5ms inference, 1.6ms postprocess per image at shape (1, 3, 768, 1280)
2

0: 768x1280 1 person, 75.6ms
Speed: 5.6ms preprocess, 75.6ms inference, 1.6ms postprocess per image at shape (1, 3, 768, 1280)
3

0: 768x1280 2 persons, 75.4ms
Speed: 5.5ms preprocess, 75.4ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
4

0: 768x1280 1 person, 75.0ms
Speed: 6.4ms preprocess, 75.0ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
5

0: 768x1280 1 person, 75.4ms
Speed: 7.0ms preprocess, 75.4ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
6

0: 768x1280 1 person, 75.1ms
Speed: 5.6ms preprocess, 75.1ms inference, 1.5ms postproces

/tmp/ipykernel_34/863348455.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'person_number': person_number, 'frame_number': frame_count, 'side': ['right', 'left'], 'angle': [right_angle, left_angle]})], ignore_index=True)


0: 768x1280 1 person, 75.2ms
Speed: 5.5ms preprocess, 75.2ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
113
113, empty, x:0, y:0

0: 768x1280 1 person, 75.6ms
Speed: 6.6ms preprocess, 75.6ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
114
114, BENT ? 64.4400348281762, 72.81429385577523

0: 768x1280 1 person, 75.2ms
Speed: 5.5ms preprocess, 75.2ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
115
115, BENT ? 75.5640250210248, 96.34019174590992

0: 768x1280 1 person, 75.6ms
Speed: 5.5ms preprocess, 75.6ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
116
116, BENT ? 90.84252426074043, 100.61965527615513

0: 768x1280 1 person, 75.3ms
Speed: 6.7ms preprocess, 75.3ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)
117
117, BENT ? 106.26020470831197, 113.40468955103232

0: 768x1280 1 person, 75.2ms
Speed: 5.6ms preprocess, 75.2ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1

In [7]:
cap = cv2.VideoCapture(video_path)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_frames

7608